In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
#starting a spark session
spark=SparkSession.builder.appName('ML').getOrCreate()

In [3]:
spark

In [4]:
train=spark.read.csv(r"C:\Users\kunal\Desktop\Github projects\test1.csv",header=True, inferSchema=True)

In [5]:
train.show()

+---------+---+----------+------+----+
|     Name|age|Experience|Salary| _c4|
+---------+---+----------+------+----+
|    Krish| 31|        10| 30000|null|
|Sudhanshu| 30|         8| 25000|null|
|    Sunny| 29|         4| 20000|null|
|     Paul| 24|         3| 20000|null|
|   Harsha| 21|         1| 15000|null|
|  Shubham| 23|         2| 18000|null|
+---------+---+----------+------+----+



In [7]:
train=train.drop('_c4')

In [8]:
train.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
#grouping the independent variables/features in a single vector [Age, Experience]
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [10]:
output=featureassembler.transform(train)

In [11]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
finalized_data=output.select('Independent Features', 'Salary')

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [15]:
### Coefficients
regressor.coefficients

DenseVector([-263.7076, 1767.624])

In [16]:
### Intercepts
regressor.intercept

19919.060052212404

In [18]:
### Prediction
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [29.0,4.0]| 20000|19342.03655352618|
+--------------------+------+-----------------+



In [19]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(657.9634464738192, 432915.89689570636)